In [ ]:

# coding: utf-8

# In[1]:


import numpy as np
import os
from imageio import imread
from skimage.transform import resize
#from scipy.misc import imread, imresize
import datetime,time
import os
import matplotlib.pyplot as plt


# We set the random seed so that the results don't vary drastically.

# In[2]:


np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)


# In this block, you read the folder names for training and validation. You also set the `batch_size` here.
# Note that you set the batch size in such a In this block, you read the folder names for training and validation. 
# You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in 
# full capacity. You keep increasing the batch size until the machine throws an error.

# In[3]:


now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

# Initializing common variables
train_path  = './Project_data/train'
val_path    = './Project_data/val'
num_epochs  = 10 # choose the number of epochs
num_classes = 5

train_doc   = np.random.permutation(open('./Project_data/train.csv').readlines())
#train_doc  = train_doc[::4]

val_doc     = np.random.permutation(open('./Project_data/val.csv').readlines())
#val_doc    = val_doc[::4]

batch_size  = 32 #experiment with the batch size
#batch_size  = 40 #experiment with the batch size
print("# training docs:",len(train_doc))
print("Batch size = ",batch_size)


# ## Generator
# This is one of the most important part of the code. The overall structure of the generator has been given.
# In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well 
# as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get 
# high accuracy.

# In[4]:


c=3
y=100
z=100 

input_shape = (30, 100,100, c)
#input_shape = (6, 120, 120, c)
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,30)]
      
    x=len(img_idx)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    newimg=resize(image,(y,z))
                    batch_data[folder,idx,:,:,0] = newimg[:,:,0] - np.percentile(newimg[:,:,0],95)/(np.percentile(newimg[:,:,0],95)- np.percentile(newimg[:,:,0],5))#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = newimg[:,:,1] - np.percentile(newimg[:,:,1],95)/(np.percentile(newimg[:,:,1],95)- np.percentile(newimg[:,:,1],5))#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = newimg[:,:,2] - np.percentile(newimg[:,:,2],95)/(np.percentile(newimg[:,:,2],95)- np.percentile(newimg[:,:,2],5))#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            
            #print("Batch: ",num_batches+1,"Index:", batch_size)
            batch=num_batches
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                                 
                    newimg=resize(image,(y,z))
                    
                    batch_data[folder,idx,:,:,0] = newimg[:,:,0] - np.percentile(newimg[:,:,0],95)/(np.percentile(newimg[:,:,0],95)- np.percentile(newimg[:,:,0],5))#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = newimg[:,:,1] - np.percentile(newimg[:,:,1],95)/(np.percentile(newimg[:,:,1],95)- np.percentile(newimg[:,:,1],5))#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = newimg[:,:,2] - np.percentile(newimg[:,:,2],95)/(np.percentile(newimg[:,:,2],95)- np.percentile(newimg[:,:,2],5))#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do            


# Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

# In[5]:


curr_dt_time = datetime.datetime.now()
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
#num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)


# In[6]:


from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#rate=0.001
rate=0.0001
optimiser = optimizers.Adam(lr=rate)#write your optimizer
#optimiser = optimizers.SGD(lr=rate)#write your optimizer
print("Learning rate = ", rate)


# ## Model
# Here you make the model using different functionalities that Keras provides. Remember to use Conv3D and MaxPooling3D and not Conv2D and Maxpooling2D for a 3D convolution model. You would want to use TimeDistributed while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

# ## Model 1 - Conv3D
# Model is built using 3 layers of Conv3D, BatchNormalization and Maxpooling3D followed by Dense Layer and output layer with following layers
#     - 1st layer Conv3D with 16 filters, kernal size = (3x3x3) and relu activation
#     - 2nd layer Conv3D with 32 filters, kernal size = (3x3x3) and relu activation
#     - 3rd layer Conv3D with 64 filters, kernal size = (3x3x3) and relu activation
#     - dense layer with 128 units and relu activation
#     used dropout of 0.5

# In[7]:


def conv3d_3C_1D_maxpooling(num_classes, input_shape):
    #write your model here
    
    # Input Layer
    model = Sequential()
    
    # 1st Layer - Conv3D with 16 filters
    model.add(Conv3D(16, (3,3,3), padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2,2,2)))
   

    # 2rd Layer - Conv3D with 32 filters
    model.add(Conv3D(32, (3,3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2,2,2)))
    

    # 3rd Layer - Conv3D with 64 filters
    model.add(Conv3D(64, (3,3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(3,3,3)))
   
    #Falttening the layers
    model.add(Flatten())

    # Adding dense layer 1
    model.add(Dense(128)) 
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    return model


# ### MODEL 2 - 2 Layer CON3D
# Mode2 is built using 2 layers of Conv3D, BatchNormalization and Maxpooling3D followed by Dense Layer and output layer with following layers
#     - 1st layer Conv3D with 32 filters, kernal size (3x3x3)  and relu activation
#     - 2nd layer Conv3D wtih 32 filters, kernal size (3x3x3)  and relu activation
#     - dense layer with 64 units and relu activation
#     
#     used frop out of 0.5

# In[8]:


def conv3d_2C_1D_maxpooling(num_classes, input_shape):
    #write your model here
    
    # Input Layer
    model = Sequential()
    
    # 1st Layer - Conv3D
    model.add(Conv3D(32, (3,3,3), padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(3,3,3)))

    # 2rd Layer - Conv3D
    model.add(Conv3D(32, (3,3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(3,3,3)))
    
    #Falttening the layers
    model.add(Flatten())

    # Adding dense layer 1
    model.add(Dense(64)) 
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    return model


# ## Model 3 - CNN+GRU
# Mode3 is built using 3 layers of Conv2D, BatchNormalization and Maxpooling3D followed by Dense Layer and  GRU layer abd an output layer with following layers
#     - 1st layer Conv2D wtih 8 filters,kernal size (3x3)  and relu activation
#     - 2nd layer Conv2D with 16 filters, kernal size (3x3) and relu activation
#     - 3nd layer Conv2D with 32 filters, kernal size (3x3) and relu activation
#     - dense layer with 128 units and relu activation
#     followed by GRU and softmax layer

# In[9]:


# Building new model with 3 layers for CNN and passing the output into GRU with fully connected layers
   
def cnn_gru(num_classes, input_shape):
  
   model_3=Sequential()

   model_3.add(TimeDistributed(Conv2D(8, (3, 3),strides=(3,3)), input_shape=input_shape))
   model_3.add(TimeDistributed(Activation("relu")))
   model_3.add(TimeDistributed(BatchNormalization()))
   model_3.add(TimeDistributed(MaxPooling2D(pool_size=2)))

   model_3.add(TimeDistributed(Conv2D(16, kernel_size =(3,3))))
   model_3.add(TimeDistributed(Activation("relu")))
   model_3.add(TimeDistributed(BatchNormalization()))
   model_3.add(TimeDistributed(MaxPooling2D(pool_size=2)))
   

   model_3.add(TimeDistributed(Conv2D(32, kernel_size =(3,3))))
   model_3.add(TimeDistributed(Activation("relu")))
   model_3.add(TimeDistributed(BatchNormalization()))
   model_3.add(TimeDistributed(MaxPooling2D(pool_size=2)))
   
              
   model_3.add(TimeDistributed(Flatten()))
   model_3.add(TimeDistributed(Dense(64,activation='relu')))
   model_3.add(Dropout(0.5))
  
   #model_3.add(GRU(512))
   #model_3.add(GRU(128))
   model_3.add(GRU(1024, return_sequences=False, dropout=0.5, go_backwards=False, stateful=False, input_shape=input_shape))
   
   model_3.add(Dense(num_classes))
   model_3.add(Activation("softmax"))
   
   return model_3


# ## Model 4 - CNN+LSTM
# 
# 
# Mode3 is built using 3 layers of Conv2D, BatchNormalization and Maxpooling3D followed by Dense Layer and  LSTM layer abd an output layer with following layers
#     - 1st layer Conv2D wtih 64 filters,kernal size (3x3)  and relu activation
#     - 2nd layer Conv2D with 64 filters, kernal size (3x3) and relu activation
#     - dense layer with 64 units and relu activation
#     followed by LSTM and ouput softmax layer

# In[10]:


# Building new model with 2 layers for CNN and passing the output into LSTM with fully connected layers
def cnn_lstm(num_classes, input_shape):
    
    model_3=Sequential()

    model_3.add(TimeDistributed(Conv2D(64, (3, 3),strides=(3,3)), input_shape=input_shape))
    model_3.add(TimeDistributed(Activation("relu")))
    model_3.add(TimeDistributed(BatchNormalization()))
    model_3.add(TimeDistributed(MaxPooling2D(pool_size=2)))

    model_3.add(TimeDistributed(Conv2D(64, kernel_size =(3,3))))
    model_3.add(TimeDistributed(Activation("relu")))
    model_3.add(TimeDistributed(BatchNormalization()))
    model_3.add(TimeDistributed(MaxPooling2D(pool_size=2)))
    

    model_3.add(TimeDistributed(Flatten()))
    model_3.add(TimeDistributed(Dense(64,activation='relu')))
    model_3.add(Dropout(0.5))
    
    model_3.add(LSTM(1024, return_sequences=False, dropout=0.5, go_backwards=False, stateful=False, input_shape=input_shape))
     
    model_3.add(Dense(num_classes, activation='softmax'))
    
    return model_3


# Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

# Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

# In[11]:


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)


# The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make. 
# Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

# In[12]:


def train_model(batch_size, model, train_generator, val_generator, num_epochs, optimiser=None):
    start_time = datetime.datetime.now()
    print(start_time)

    num_train_sequences = len(train_doc)
    print('\n# training sequences =', num_train_sequences)

    num_val_sequences = len(val_doc)
    print('# validation sequences =', num_val_sequences)
    print('# epochs =', num_epochs)
    print('\n')
    
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print (model.summary())
    
    # Calculating steps_per_epoch for training set
    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1
    
    # Calculating steps_per_epoch for validation set
    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1    
    
    
    # Saving the model to disk
    model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
    if not os.path.exists(model_name):
        os.mkdir(model_name)
    
    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
    
    #LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)## write the REducelronplateau code here
    LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
    callbacks_list = [checkpoint, LR]
    
    history=model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
    
    end_time = datetime.datetime.now()
    difference = end_time-start_time
    print("\n\nTraining time (in seconds): ", difference)
    
    return history


# In[13]:


def plot_acc(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


#   #### calling the first model  with 3 conv3d layers

# In[14]:


conv3d_model_1 = conv3d_3C_1D_maxpooling(num_classes, input_shape)
history = train_model(batch_size, conv3d_model_1, train_generator, val_generator, num_epochs, optimiser)


# #### plot the accuracy for the first model conv-3d 

# In[15]:


plot_acc(history)


#   #### calling the second model conv-3d with 2 layers

# In[16]:


#conv3d_model_2 = conv3d_2C_1D_maxpooling(num_classes, input_shape)
#history = train_model(batch_size, conv3d_model_2, train_generator, val_generator, num_epochs, optimiser)


# #### plot the accuracy for the second model conv-3d 

# In[17]:


#plot_acc(history)


# #### calling the Third model with two layers of conv-2d  followed by GRU

# In[18]:


#cnn_rnn_model_3 = cnn_gru(num_classes, input_shape)
#history = train_model(batch_size, cnn_rnn_model_3, train_generator, val_generator, num_epochs, optimiser)


# #### plot the accuracy for the Third  model  of CNN+GRU

# In[19]:


#plot_acc(history)


# #### calling the forth model with two layers of conv-2d  followed by LSTM

# In[20]:


#cnn_rnn_model_4 = cnn_lstm(num_classes, input_shape)
#history = train_model(batch_size, cnn_rnn_model_4, train_generator, val_generator, num_epochs, optimiser)


# #### plot the accuracy for the Forth  model  of CNN+LSTM

# In[21]:


#plot_acc(history)

